# Data Extraction and EDA: State Weather Data

In [27]:
import sys
# Append path to import helper modules
sys.path.append("../")


import pandas as pd
import pickle
import datetime
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from utilities import states_to_abbrev, abbrev_to_states

In [18]:
"""
There's a ton of columns, but after reading some stuff up, it looks like
the "dry bulb" temperature is the temperature that is meaningful.
"""
columns = ["STATION", "DATE", "REPORT_TYPE", "SOURCE", "AWND", "BackupDirection", "BackupDistance", "BackupDistanceUnit", "BackupElements", "BackupElevation", "BackupElevationUnit", "BackupEquipment", "BackupLatitude", "BackupLongitude", "BackupName", "CDSD", "CLDD", "DSNW", "DailyAverageDewPointTemperature", "DailyAverageDryBulbTemperature", "DailyAverageRelativeHumidity", "DailyAverageSeaLevelPressure", "DailyAverageStationPressure", "DailyAverageWetBulbTemperature", "DailyAverageWindSpeed", "DailyCoolingDegreeDays", "DailyDepartureFromNormalAverageTemperature", "DailyHeatingDegreeDays", "DailyMaximumDryBulbTemperature", "DailyMinimumDryBulbTemperature", "DailyPeakWindDirection", "DailyPeakWindSpeed", "DailyPrecipitation", "DailySnowDepth", "DailySnowfall", "DailySustainedWindDirection", "DailySustainedWindSpeed", "DailyWeather", "HDSD", "HTDD", "HeavyFog", "HourlyAltimeterSetting", "HourlyDewPointTemperature", "HourlyDryBulbTemperature", "HourlyPrecipitation", "HourlyPresentWeatherType", "HourlyPressureChange", "HourlyPressureTendency", "HourlyRelativeHumidity", "HourlySeaLevelPressure", "HourlySkyConditions", "HourlyStationPressure", "HourlyVisibility", "HourlyWetBulbTemperature", "HourlyWindDirection", "HourlyWindGustSpeed", "HourlyWindSpeed", "MonthlyAverageRH", "MonthlyDaysWithGT001Precip", "MonthlyDaysWithGT010Precip", "MonthlyDaysWithGT32Temp", "MonthlyDaysWithGT90Temp", "MonthlyDaysWithLT0Temp", "MonthlyDaysWithLT32Temp", "MonthlyDepartureFromNormalAverageTemperature", "MonthlyDepartureFromNormalCoolingDegreeDays", "MonthlyDepartureFromNormalHeatingDegreeDays", "MonthlyDepartureFromNormalMaximumTemperature", "MonthlyDepartureFromNormalMinimumTemperature", "MonthlyDepartureFromNormalPrecipitation", "MonthlyDewpointTemperature", "MonthlyGreatestPrecip", "MonthlyGreatestPrecipDate", "MonthlyGreatestSnowDepth", "MonthlyGreatestSnowDepthDate", "MonthlyGreatestSnowfall", "MonthlyGreatestSnowfallDate", "MonthlyMaxSeaLevelPressureValue", "MonthlyMaxSeaLevelPressureValueDate", "MonthlyMaxSeaLevelPressureValueTime", "MonthlyMaximumTemperature", "MonthlyMeanTemperature", "MonthlyMinSeaLevelPressureValue", "MonthlyMinSeaLevelPressureValueDate", "MonthlyMinSeaLevelPressureValueTime", "MonthlyMinimumTemperature", "MonthlySeaLevelPressure", "MonthlyStationPressure", "MonthlyTotalLiquidPrecipitation", "MonthlyTotalSnowfall", "MonthlyWetBulb", "NormalsCoolingDegreeDay", "NormalsHeatingDegreeDay", "REM", "REPORT_TYPE", "SOURCE", "ShortDurationEndDate005", "ShortDurationEndDate010", "ShortDurationEndDate015", "ShortDurationEndDate020", "ShortDurationEndDate030", "ShortDurationEndDate045", "ShortDurationEndDate060", "ShortDurationEndDate080", "ShortDurationEndDate100", "ShortDurationEndDate120", "ShortDurationEndDate150", "ShortDurationEndDate180", "ShortDurationPrecipitationValue005", "ShortDurationPrecipitationValue010", "ShortDurationPrecipitationValue015", "ShortDurationPrecipitationValue020", "ShortDurationPrecipitationValue030", "ShortDurationPrecipitationValue045", "ShortDurationPrecipitationValue060", "ShortDurationPrecipitationValue080", "ShortDurationPrecipitationValue100", "ShortDurationPrecipitationValue120", "ShortDurationPrecipitationValue150", "ShortDurationPrecipitationValue180", "Sunrise", "Sunset", "TStorms", "WindEquipmentChangeDate"]
keep = set(["STATION", "REPORT_TYPE", "SOURCE", "AWND", "DailyAverageDryBulbTemperature", "HourlyDryBulbTemperature"])
columns_to_remove = set(columns).difference(keep)


In [147]:
"""
Concat all the government weather data together.
"""
RAW_DATA_DIR = "../data/raw_data/"
csv_data = []

for i in range(1, 7):
    csv_data.append(pd.read_csv(f"{RAW_DATA_DIR}state_weather_{i}.csv"))
    

/Users/Emily/opt/anaconda3/envs/cs109b/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,25,26,27,28,29,30,31,32,35,36,37,41,42,43,44,45,48,49,50,51,52,53,54,55,56,58,59,68,69,71,72,85,88,93,96,97,98,99,100,101,102,103,104,105,106,107,114,117,119,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Emily/opt/anaconda3/envs/cs109b/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,5,7,8,11,14,19,25,26,27,28,29,30,31,32,33,34,35,36,37,41,42,43,44,45,48,49,50,51,52,53,54,55,56,58,59,60,69,71,72,76,88,93,95,96,97,98,99,100,101,102,103,104,105,106,107,117,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Emily/opt/anaconda3/envs/cs109b/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

In [149]:
states_weather = pd.concat(csv_data, axis=0)
states_weather = states_weather.rename(columns={"STATION": "station_id"})
states_weather["date"] = pd.to_datetime(states_weather["DATE"], infer_datetime_format=True)
states_weather = states_weather.drop(columns=columns_to_remove)

# An entire dataset for Utah is problematic. Drop it.
states_weather = states_weather[states_weather["station_id"] != "A0001603036"]

states_weather["HourlyDryBulbTemperature"] = pd.to_numeric(
    states_weather["HourlyDryBulbTemperature"], errors="coerce"
)
states_weather = states_weather.dropna(subset=["HourlyDryBulbTemperature"])

display(states_weather)

,station_id,REPORT_TYPE,SOURCE,AWND,DailyAverageDryBulbTemperature,HourlyDryBulbTemperature,REPORT_TYPE.1,SOURCE.1,date
0,72216013869,FM-15,7,NaN,NaN,40.0,FM-15,7,2020-01-01 00:53:00
1,72216013869,FM-15,7,NaN,NaN,39.0,FM-15,7,2020-01-01 01:53:00
2,72216013869,FM-15,7,NaN,NaN,37.0,FM-15,7,2020-01-01 02:53:00
3,72216013869,FM-15,7,NaN,NaN,37.0,FM-15,7,2020-01-01 03:53:00
4,72216013869,FM-15,7,NaN,NaN,36.0,FM-15,7,2020-01-01 04:53:00
...,...,...,...,...,...,...,...,...,...
36666,99999904138,CRN05,I,NaN,NaN,47.0,CRN05,I,2020-05-06 20:40:00
36667,99999904138,CRN05,I,NaN,NaN,47.0,CRN05,I,2020-05-06 20:45:00
36668,99999904138,CRN05,I,NaN,NaN,47.0,CRN05,I,2020-05-06 20:50:00
36669,99999904138,CRN05,I,NaN,NaN,46.0,CRN05,I,2020-05-06 20:55:00


In [150]:
"""
Ughhhh. The text file for the station IDs is so inconsistently formatted,
so we can't just read it with a pandas or np and a basic delimiter.

The station metadata is global, but we want to extract 
IDS for all of the US states only.
"""

def extract_station_metadata():
    station_df = []
    station_ids = open("../data/raw_data/station_metadata.txt", "r")

    # state start index
    ss_index = 47
    wban_index = 7
    for i, line in enumerate(station_ids):
        # skip header
        if i == 0:
            continue
        state = line[ss_index: ss_index + 3].strip()
        if state.isspace():
            continue
        else:
            # It's a US state. Start parsing.
            wban_id = line[wban_index: wban_index + 6].strip()
            station_meta = line[0:7].strip()
            # Because why not?
            lattitude = line[57:66].strip()
            longitude = line[65:75].strip()
            elevation = line[74:82].strip()
            station_name = line[13:43].strip()
            station_id = f"{station_meta}{wban_id}"
            try:
                # The weather data has the station ID as an integer. 
                # Data types must match up in the join, so throw out anything that's not
                # castable to an int.
                station_id = int(station_id)
            except:
                continue
            row = {
                "wban_id": int(wban_id),
                "state": state,
                "station_meta": station_meta,
                "lattitude": lattitude,
                "longitude": longitude,
                "elevation": elevation,
                "station_name": station_name,
                "station_id": station_id
            }
            station_df.append(row)
    result = pd.DataFrame(station_df)
    return result

station_df = extract_station_metadata()
display(station_df)

,wban_id,state,station_meta,lattitude,longitude,elevation,station_name,station_id
0,99999,,007018,+00.000 +,+000.000 +,+7018.0,WXPOD 7018,701899999
1,99999,,007026,+00.000 +,+000.000 +,+7026.0,WXPOD 7026,702699999
2,99999,,007070,+00.000 +,+000.000 +,+7070.0,WXPOD 7070,707099999
3,99999,,008260,+00.000 +,+000.000 +,+0000.0,WXPOD8270,826099999
4,99999,,008268,+32.950 +,+065.567 +,+1156.7,WXPOD8278,826899999
...,...,...,...,...,...,...,...,...
29703,96405,AK,999999,+60.473 -,-145.354 +,+0025.3,CORDOVA 14 ESE,99999996405
29704,96406,AK,999999,+64.502 -,-154.130 +,+0078.9,RUBY 44 ESE,99999996406
29705,96407,AK,999999,+66.562 -,-159.004 +,+0006.7,SELAWIK 28 E,99999996407
29706,96408,AK,999999,+63.452 -,-150.875 +,+0678.2,DENALI 27 N,99999996408


In [151]:
states_df = states_weather.merge(station_df, left_on="station_id", right_on="station_id")

In [152]:
"""
Now, let's check to be sure there's data for all states. Hopefully we are not missing any!
"""
grouped_states = states_df.groupby(["station_name", "station_id", "state"]).agg(
    {"station_name": "first", "station_id": "first", "state": "first"}
).reset_index(drop=True)
states_list = set(grouped_states["state"].values)

all_states = [
    'AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'GU', 'HI', 
    'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 
    'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'MP', 'OH', 'OK', 'OR', 'PA', 
    'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY'
]

missing_states = set(all_states).difference(states_list)
ms_full = [abbrev_to_states[x] for x in list(missing_states)]
print(ms_full)

['District of Columbia', 'Puerto Rico']


In [170]:
"""
Missing states above. 

There are some errors with the datasets from GOV, marked with a "warning" sign, so
I will re-investigate. Since I had to download these states one-by-one, will investigate after I get an initial weather 
dataset crunched.
"""
cleaned_weather_data = states_df.groupby(["state", states_df["date"].dt.year, states_df["date"].dt.month, states_df["date"].dt.day]).agg({
    "state": "first",
    "date": "first",
    "HourlyDryBulbTemperature": "mean"
}).rename(columns={"HourlyDryBulbTemperature": "average_temperature"}).reset_index(drop=True)


In [172]:
display(cleaned_weather_data)

,state,date,average_temperature
0,AK,2020-01-01 00:42:00,29.145161
1,AK,2020-01-02 00:56:00,26.428571
2,AK,2020-01-03 01:56:00,23.725000
3,AK,2020-01-04 00:04:00,33.174603
4,AK,2020-01-05 00:29:00,27.931034
...,...,...,...
6665,WY,2020-05-02 00:29:00,42.972973
6666,WY,2020-05-03 00:53:00,44.666667
6667,WY,2020-05-04 00:53:00,42.458333
6668,WY,2020-05-05 00:53:00,42.500000


In [174]:
# dump to pickle file
pickle.dump(cleaned_weather_data, open("../data/daily_average_temp_by_state.p", "wb" ))